In [ ]:
#this query will calculate the frquency of antihypertensive prescribing at the point of admission in the care home cohort. 
#call all medication for the care home cohort
create or replace table `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_antihypertensive_admission` as (
with care_home_medication as (
  select 
  *,
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srprimarycaremedication`
  where person_id in (select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)
),
#call the bnf code and dmd(snomed) code 
e as (select BNF_Code,BNF_Name, cast(SNOMED_CODE as string) as SNOMEDCODE_STRING
  from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_BNF_DMD_SNOMED_lkp`),

# map the dmd code in primary care table to the bnf code in the lookup table to enable the antihypertensives to be identified based on the bnf sub sub chapter
f as (select 
d.*,
e.BNF_Code,
e.BNF_Name
from care_home_medication d 
left join e
on d.idmultilexdmd = e.SNOMEDCODE_STRING),

medication_join_admission as (
  select
  f.*,
  c.first_episodestartdate
  from f 
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` c
  on f.person_id = c.person_id
),
#select antihypertensive based on the bnf code
g as (
select 
SUBSTRING(BNF_code,1,7) as BNF_sub_sub_chapter,
person_id,
datemedicationstart,
datemedicationend,
medicationdosage, 
medicationquantity,
isrepeatmedication,
first_episodestartdate,
BNF_code,
BNF_name,
from medication_join_admission
where (BNF_Code like '0206020%') or (BNF_Code like '0204000%') or (BNF_Code like '020201%') or (BNF_code like  '020505%') or (BNF_code like '020504%')
order by person_id),
#select repeat medication prescribed at the point of admission (14 days either side)
meds_admission as (select
*
from g
where datemedicationstart between datetime_sub(first_episodestartdate, INTERVAL 14 day) and datetime_add(first_episodestartdate,interval 14 day)),
#aggregate by antihypertensive type (based on bnf sub sub chapter)as only interested in whether the medication was prescribed once during peruiod
mad_2 as(
select
person_id,
count(*), bnf_sub_sub_chapter, 
from meds_admission
where isrepeatmedication = 'true'
group by person_id,bnf_sub_sub_chapter
order by person_id),

mad_array as(
select
person_id,
 ARRAY_AGG(DISTINCT BNF_sub_sub_chapter) AS antihypertensive
from mad_2
group by person_id)

select
person_id,
IF('0206020' IN UNNEST(mad_array.antihypertensive), 'true', 'false') AS calcium_channel_blocker,
IF('0204000' IN UNNEST(mad_array.antihypertensive), 'true', 'false') AS beta_blocker,
IF('0202010' IN UNNEST(mad_array.antihypertensive), 'true', 'false') AS thiazide_type_diuretic,
IF('0205052' IN UNNEST(mad_array.antihypertensive),'true','false') as angiotensin_receptor_blocker,
IF('0205051' IN UNNEST(mad_array.antihypertensive),'true','false') as ace_inhibitor,
IF('0205040' IN UNNEST(mad_array.antihypertensive),'true','false') as alpha_blocker,
array_length(mad_array.antihypertensive) as no_antihypertensive
from mad_array
order by person_id)